## Softmax regression with mini-batch gradient descent

First, let's import the dependencies. Since we're building everything from scratch, we'll be using numpy and sklearn's `datasets` module for loading the iris dataset.

In [465]:
import numpy as np
from sklearn import datasets

In [466]:
iris = datasets.load_iris()
list(iris.keys())

['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename']

Let's get the independent and depedent data, keeping in mind that we need to initialize a column of ones before any other columns in `X`, for the bias terms.  

In [467]:
X = iris["data"]
X_with_bias = np.c_[np.ones((len(X),1)), X]
y = iris["target"]

In [468]:
num_classes = y.max() + 1

Next, let's define some useful functions that we'll be using throughout the session:

In [469]:
def one_hot_enc(target):
  one_hot = np.zeros((target.size, num_classes))
  one_hot[np.arange(target.size), target] = 1
  return one_hot

In [470]:
def softmax(x): 
  exp = np.exp(x)
  return exp / np.expand_dims(np.sum(exp, axis=1), 1)

In [471]:
def init_theta(): return np.random.rand(X_with_bias.shape[1], num_classes)

This is a trick that is frequently used in machine learning called `logsumexp`, which is a more efficient way of computing the log of softmax. `epsilon` is a small numerical constant added to ensure that the thing that we are taking the `log` of is not 0 (which would result in `nan`). Read more about `logsumexp` [here](https://blog.feedly.com/tricks-of-the-trade-logsumexp/). There are also numerically stable versions of `logsumexp` but this is not implemented here for simplicity.

In [472]:
def LogSumExp(y, epsilon):
  return y - np.log(np.exp(y).sum(axis = 1, keepdims=True) + epsilon)

Below is a function that generates mini-batches that I've taken from a [stackoverflow](https://stackoverflow.com/questions/38157972/how-to-implement-mini-batch-gradient-descent-in-python) thread.

In [473]:
def iterate_minibatches(x,y,bs,shuffle=True):
  assert x.shape[0] == y.shape[0]
  if shuffle:
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices) # shuffles in-place
  for start_idx in range(0, x.shape[0], bs):
    end_idx = min(start_idx + bs, x.shape[0])
    if shuffle:
      idx = indices[start_idx: end_idx]
    else:
      idx = slice(start_idx, end_idx)
    yield x[idx], y[idx]

Since we are not using `sklearn` for anything besides loading the dataset, we also create test, validation and training sets manually. 

In [474]:
test_pct = 0.2
val_pct = 0.2
n_test = int(test_pct * m)
n_val = int(val_pct * m)
n_train = m - n_test - n_val

random_indices = np.random.permutation(m)
test_indices = random_indices[:n_test]
val_indices = random_indices[n_test:n_test+n_val]
train_indices = random_indices[-n_train:]

X_train, X_val, X_test = X_with_bias[train_indices], X_with_bias[val_indices], X_with_bias[test_indices]
assert X_train.shape[0] == n_train and X_val.shape[0] == n_val and X_test.shape[0] == n_test
y_train, y_val, y_test = y[train_indices], y[val_indices], y[test_indices]
assert y_train.shape[0] == n_train and y_val.shape[0] == n_val and y_test.shape[0] == n_test

Now for our main training loop:

First, we declare some hyperparameters such as the learning rate (`lr`), number of epochs, batch size (`bs`), and `alpha`, which is the l2 regularisation constant.

For each epoch, we iterate through all our mini-batches, get the prediction, and use the prediction and target values to compute the gradient, remembering that we add an additional l2 regularisation term at the end.

We also calculate the loss on the validation set to evaluate performance and print it out every 500 epochs.

In [483]:
#Hyperparams
lr = 0.1
n_epochs = 5001
m = X.shape[0]
bs = 10 # batch size, so for 150 samples, we need to divide it into 15 groups of 10 instances each
n_batches = m//bs
epsilon = 1e-7 # to avoid nan results from log(0)
alpha = 0.01 # l2 regularisation 
min_loss = np.inf

theta = init_theta()

for epoch in range(n_epochs):
  for batch in iterate_minibatches(X_train, one_hot_enc(y_train), bs):
    xb_train, yb_train = batch
    preds = softmax(xb_train@theta)
    gradients = (1/xb_train.shape[0]) * (xb_train.T@(preds-yb_train)) + np.vstack([np.zeros((1,num_classes)), alpha * theta[1:]])
    theta = theta - lr * gradients

  y_val_pred = LogSumExp(X_val@theta, epsilon) 
  cross_entropy_loss = -np.mean(y_val_pred[np.arange(X_val.shape[0]), y_val]) 
  l2_loss = alpha * 1/2 * np.square(theta[1:]).sum()
  total_loss = cross_entropy_loss + l2_loss

  if epoch % 500 == 0: 
    print(f"Current loss: {total_loss}")
 

Current loss: 0.9559835133592577
Current loss: 0.2835619290359492
Current loss: 0.21752716209131
Current loss: 0.23637013398738982
Current loss: 0.21555417989668607
Current loss: 0.2069807318737825
Current loss: 0.20279665261068736
Current loss: 0.20305770855358207
Current loss: 0.23359282432556988
Current loss: 0.2317153779434589
Current loss: 0.21075293023173117


Now, let's see what the final model parameters that we've trained are: 

In [484]:
theta

array([[ 8.44033927,  2.80588604, -9.26926123],
       [-0.16791993,  0.45902131, -0.29110137],
       [ 0.99125651, -0.16769693, -0.82355958],
       [-2.36250324, -0.15828285,  2.52078609],
       [-1.02448477, -0.99657442,  2.02105919]])

And finally, let's use these values to run predictions on the test set and evaluate the performance. 

In [487]:
preds = np.argmax(X_test@theta, axis=1)

In [488]:
np.mean(preds == y_test)

1.0

100% accuracy ain't bad! That said, this value changes from run to run and may be different when you run the notebook yourself. On my end, however, we seem to be getting at least 95% accuracy consistently. 